# RNN for classifying clade 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable 

torch.manual_seed(123)

k=28

data = pd.read_csv ("/Users/jarodw/Documents/pathogenetic-potential/DATA.csv")

df = pd.DataFrame(data, columns = ['strain', 'seq', 'clade'])

clades = pd.unique(df['clade'])
print(clades)

clade_count = df["clade"].value_counts()
clade_count.plot(kind='barh')

# K-Mer Sequence

In [ ]:
def Kmers_funct(seq, size):
    return [seq[x:x+size].lower() for x in range(len(seq) - size + 1)]

sentences = []
for j in range(len(df)) :
    sequence = str(df.loc[j, "seq"])
    sentence = ' '.join(Kmers_funct(sequence, size=k))
    sentences.append(sentence)
    
    
df['sentence'] = sentences    
    
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(28,28))

print(df.head())

X = cv.fit_transform(sentences)

print('Shape of data:', X.shape)

Y = pd.get_dummies(df['clade']).values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.20,random_state=42)

y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test))


X_train_tensor = torch.from_numpy(X_train.todense()).float()
X_train_tensor_final = torch.reshape(X_train_tensor,   (X_train_tensor.shape[0], 1, X_train_tensor.shape[1]))
X_test_tensor = torch.from_numpy(X_test.todense()).float()
Y_train_tensor = torch.from_numpy(np.array(y_train))
Y_train_tensor = Y_train_tensor.to(torch.float32)

Y_test_tensor = torch.from_numpy(np.array(y_test))
Y_test_tensor = Y_test_tensor.to(torch.float32)

print(X_train_tensor.size())
print(X_train_tensor_final.size())
print(Y_train_tensor.size())
print(Y_train_tensor)

In [ ]:
#Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
        self.fc = nn.Linear(128, num_classes) #fully connected last layer
       

        self.relu = nn.ReLU()
        
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc(out) 
         #Final Output
        return out

In [ ]:
num_epochs = 100 #1000 epochs
learning_rate = 0.001 #0.001 lr

input_size = 1048857 #number of features
hidden_size = 2 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

num_classes = 10 #number of output classes
 
lstm1 = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensor_final.shape[1]) #our lstm class 

In [ ]:
criterion = torch.nn.CrossEntropyLoss()   # cross entropy for categorical response
optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate) 


In [ ]:
for epoch in range(num_epochs):
  outputs = lstm1.forward(X_train_tensor_final) #forward pass
  optimizer.zero_grad() #caluclate the gradient, manually setting to 0
 
  # obtain the loss function
  loss = criterion(outputs, Y_train_tensor)
 
  loss.backward() #calculates the loss of the loss function
 
  optimizer.step() #improve from loss, i.e backprop
  if epoch % 1 == 0:
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 